In [ ]:
%pip install langchain langchain-openai langchain-core

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime
import random
import pandas as pd
import numpy as np
import json

from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("OpenAI API Key not found")

# Tools

In [ ]:
@tool
def get_dataframe_info() -> str:
    """Get basic information about the DataFrame including shape, columns, and data types."""
    info = {
        "shape": df.shape,
        "columns": list(df.columns),
        "dtypes": {col: str(dtype) for col, dtype in df.dtypes.items()},
        "memory_usage": f"{df.memory_usage(deep=True).sum()} bytes"
    }
    return json.dumps(info, indent=2)

@tool
def get_column_stats(column_name: str) -> str:
    """Get statistical summary for a specific column."""
    if column_name not in df.columns:
        return f"Column '{column_name}' not found. Available columns: {list(df.columns)}"
    
    col = df[column_name]
    
    if col.dtype in ['int64', 'float64']:
        stats = {
            "count": col.count(),
            "mean": col.mean(),
            "std": col.std(),
            "min": col.min(),
            "25%": col.quantile(0.25),
            "50%": col.median(),
            "75%": col.quantile(0.75),
            "max": col.max()
        }
    else:
        stats = {
            "count": col.count(),
            "unique": col.nunique(),
            "top": col.mode().iloc[0] if not col.mode().empty else "N/A",
            "freq": col.value_counts().iloc[0] if len(col.value_counts()) > 0 else 0
        }
    
    return json.dumps(stats, indent=2)

@tool
def get_missing_values() -> str:
    """Check for missing values in the DataFrame."""
    missing = df.isnull().sum()
    missing_dict = {col: int(count) for col, count in missing.items()}
    total_missing = missing.sum()
    
    return json.dumps({
        "total_missing_values": int(total_missing),
        "missing_by_column": missing_dict,
        "percentage_missing": {col: round((count/len(df))*100, 2) for col, count in missing_dict.items()}
    }, indent=2)

# Code

In [ ]:
# Create a sample DataFrame for testing
sample_data = {
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'age': [25, 30, 35, 28, 22],
    'salary': [50000, 75000, 90000, 60000, 45000],
    'department': ['Engineering', 'Sales', 'Engineering', 'Marketing', 'Sales'],
    'years_experience': [3, 8, 12, 5, 1]
}
df = pd.read_csv("../datasets/smoke.csv")
print(df.head())

In [ ]:
# Initialize GPT-4o-mini with tool calling enabled
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    api_key=os.environ["OPENAI_API_KEY"]
)

# Check if the model supports tool calling
print(f"Model: {llm.model_name}")
print(f"Supports tool calling: {hasattr(llm, 'bind_tools')}")

In [ ]:
# Collect all tools imported from pandas_tools.ipynb (doesn't work)
# tools = pandas_tools.tools

# Local tools
tools = [get_dataframe_info, get_column_stats, get_missing_values]

In [ ]:
# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print("Available pandas tools:")
for tool_func in tools:
    print(f"- {tool_func.name}: {tool_func.description}")

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful data analysis assistant. You have access to pandas DataFrame tools that can help analyze data.
    
    The current DataFrame contains employee information with columns: name, age, salary, department, years_experience.

    When users ask about data analysis, use the appropriate tools to get the information they need.
    Always provide clear, helpful explanations of the results."""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [ ]:
# Create the agent using create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,  # Shows which tools are being called
    handle_parsing_errors=True
)

print("Agent and AgentExecutor created successfully!")

In [ ]:
response = agent_executor.invoke({"input": "What's the shape and structure of the DataFrame?"})
print(response['output'])

response = agent_executor.invoke({"input": "Are there any missing values in the dataset?"})
print(response['output'])